In [32]:
import os
from google import genai

In [33]:
LLM_API_KEY = os.environ.get('GEMINI_API_KEY')
if not LLM_API_KEY:
    raise ValueError("Please set the environment variable 'GEMINI_API_KEY' with your API key.")


In [34]:
client = genai.Client()
client.api_key = LLM_API_KEY

In [35]:
from minsearch import Index
import json


with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [36]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [37]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [38]:
index = Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course'],
)

q = "the course is already started, can I still enroll in the course?"

In [39]:
index.fit(documents)

In [40]:

def search(query):

    boost = {'question': 3.0, 'section': 0.5}

    return index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
        )




In [41]:
def call_gemini(q: str):
    respose = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=q  
    )
    return respose


In [42]:
prompt_template = """
You are a helpful assistant that answers questions based on the provided context.
If the context does not contain enough information to answer the question, respond with "I don't know".
You will be given a question and some context. Use the context to answer the question.
Here are the context:
{context}
Now, answer the question:
{question}
""".strip()


In [43]:


def build_prompt(question, search_results) -> str:
    context = ""

    for result in search_results:
        context += f"Section: {result['section']}\nQuestion: {result['question']}\nAnswer: {result['text']}\n\n"


    return prompt_template.format(context=context, question=question).strip()



In [44]:
def run_rag(query: str):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    response = call_gemini(prompt)
    return response.text

In [45]:
print(run_rag("Can I enroll in the course after it has started?"))
print("Done!")


Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Done!


In [46]:
result = run_rag("how to install kafka?")
print(result)

The provided context includes instructions for installing `kafka-python-ng` using pip, running Java Kafka producers/consumers from the terminal, and resolving "ModuleNotFoundError: No module named 'kafka'". However, it does not contain general instructions for installing Kafka itself (the distributed streaming platform).



In [47]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")

In [48]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [49]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [50]:
from tqdm.auto import tqdm

In [51]:
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [52]:
query = "the couse is already started, can I still enroll in the course?"

def es_search(query: str):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    es_resp = es_client.search(index=index_name, body=search_query)

    return [hit['_source'] for hit in es_resp['hits']['hits']]   

In [53]:
def run_rag(query: str):
    search_results = es_search(query)
    prompt = build_prompt(query, search_results)
    response = call_gemini(prompt)
    return response.text

In [54]:
run_rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n"

In [ ]:
query = "How do execute a command on a Kubernetes pod?"
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
es_resp = es_client.search(index=index_name, body=search_query)
print(es_resp)

{'took': 23, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'Z9h_l5cBNhPPY2UvhggO', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': '9th_l5cBNhPPY2UviAi2', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard', '

In [59]:
# es_client.indices.delete(index=index_name)
query = "How do copy a file to a Docker container?"
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
es_resp = es_client.search(index=index_name, body=search_query)
print(es_resp)

{'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 340, 'relation': 'eq'}, 'max_score': 73.38676, 'hits': [{'_index': 'course-questions', '_id': 'Z9h_l5cBNhPPY2UvhggO', '_score': 73.38676, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'hth_l5cBNhPPY2UvhgiE', '_score': 66.688705, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or dire

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()


prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

search_results = [hit['_source'] for hit in es_resp['hits']['hits']]


In [71]:
def build_prompt_hm(question, search_results) -> str:
    context = ""

    for result in search_results:
        # context += f"Question: {result['question']}\nAnswer: {result['text']}\n\n"
        context += context_template.format(question=result['question'], text=result['text']) + "\n\n"


    return prompt_template.format(context=context, question=question).strip()

In [72]:
prompt = build_prompt_hm(query, search_results)
print(f"prompt len: {len(prompt)}")
print(prompt)

prompt len: 1446
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id

In [80]:
response = call_gemini(prompt)


In [91]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax is as follows:\n\n`docker cp /path/to/local/file_or_directory container_id:/path/in/container`\n\nAlternatively, in the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\n\n`COPY ["src/predict.py", "models/xgb_model.bin", "./"]`\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, url_context_metadata=None, avg_logprobs=-0.029048277952960718, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_versi

In [92]:
print(f"token count: {response.usage_metadata.total_token_count}")
print(response.text)

token count: 460
You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax is as follows:

`docker cp /path/to/local/file_or_directory container_id:/path/in/container`

Alternatively, in the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:

`COPY ["src/predict.py", "models/xgb_model.bin", "./"]`



In [94]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")


length = len(encoding.encode(prompt))
print(f"response token count: {length}")



response token count: 320
